In [42]:
pip install pyyaml h5py  # Required to save models in HDF5 format

In [43]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.6.0


In [44]:
(train_images, train_labels), (test_images, test_labels)=tf.keras.datasets.mnist.load_data()
train_labels=train_labels[:1000]
test_labels=test_labels[:1000]

train_images=train_images[:1000].reshape(-1, 28*28)/255.0
test_images=test_images[:1000].reshape(-1,28*28)/255.0

In [45]:
def createModel():
  model=tf.keras.models.Sequential([
      keras.layers.Dense(512, activation='relu', input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10)
  ])
  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])
  return model

model=createModel()
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [46]:
checkpoint_path="training_1/cp.ckpt"
checkpoint_dir=os.path.dirname(checkpoint_path)

cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                               save_weigths_only=True,
                                               verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 1s 11ms/step - loss: 1.1707 - sparse_categorical_accuracy: 0.6810 - val_loss: 0.7103 - val_sparse_categorical_accuracy: 0.7950

Epoch 00001: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets
Epoch 2/10
32/32 [==============================] - 0s 7ms/step - loss: 0.4138 - sparse_categorical_accuracy: 0.8920 - val_loss: 0.5399 - val_sparse_categorical_accuracy: 0.8280

Epoch 00002: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets
Epoch 3/10
32/32 [==============================] - 0s 8ms/step - loss: 0.2730 - sparse_categorical_accuracy: 0.9250 - val_loss: 0.4800 - val_sparse_categorical_accuracy: 0.8520

Epoch 00003: saving model to training_1/cp.ckpt
INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2059 - sparse_categorical_accuracy: 0.9480 - val_loss: 0.4443

In [47]:
os.listdir(checkpoint_dir)

['cp.ckpt']

In [48]:
model=createModel()

loss, acc=model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.2742 - sparse_categorical_accuracy: 0.1250
Untrained model, accuracy: 12.50%


In [49]:
model.load_weights(checkpoint_path)
loss, acc=model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4195 - sparse_categorical_accuracy: 0.8690
Restored model, accuracy: 86.90%


In [50]:
checkpoint_path="training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir=os.path.dirname(checkpoint_path)

batch_size=32

cp_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

model=createModel()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [51]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0030.ckpt.index']

In [52]:
latest=tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [53]:
model=createModel()
model.load_weights(latest)
loss, acc=model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4947 - sparse_categorical_accuracy: 0.8800
Restored model, accuracy: 88.00%


In [54]:
model.save_weights('./checkpoints/my_checkpoint')
model=createModel()
model.load_weights('./checkpoints/my_checkpoint')
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

Restored model, accuracy: 88.00%


In [55]:
model=createModel()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1659 - sparse_categorical_accuracy: 0.6720
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4249 - sparse_categorical_accuracy: 0.8730
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2811 - sparse_categorical_accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2227 - sparse_categorical_accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1620 - sparse_categorical_accuracy: 0.9590
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [58]:
!ls saved_model

!ls saved_model/my_model

my_model
assets	keras_metadata.pb  saved_model.pb  variables


In [60]:
new_model=tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [61]:
loss, acc=new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))
print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4190 - sparse_categorical_accuracy: 0.8640
Restored model, accuracy: 86.40%
(1000, 10)


In [62]:
model=createModel()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.2030 - sparse_categorical_accuracy: 0.6520
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4257 - sparse_categorical_accuracy: 0.8730
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2922 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2129 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1580 - sparse_categorical_accuracy: 0.9700


In [64]:
new_model=tf.keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [65]:
loss, acc=new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4478 - sparse_categorical_accuracy: 0.8580
Restored model, accuracy: 85.80%
